In [36]:
from jinja2 import FileSystemLoader, Environment
#from openpyxl import Workbook
#from openpyxl import load_workbook
from glom import glom
import numpy as np

import sys,os,re

from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

sys.path.append("/Users/szagar/ZTS/Dropbox/Business/ats/Code/Model/v3")
sys.path.append("/Users/szagar/ZTS/Dropbox/Business/ats/Code/lib")

from oos_test import OosTest
from strategy import Strategy
from candidate import Candidate
from prototype import Prototype
from proto_config import ProtoConfig
from prototype_parameter import PrototypeParameter
from entry_filter import EntryFilter
from poi import Poi
from session import Session as TradeSession
from base import Base, Session

#from ats_tools import add2hhmm
from ats_tools import hhmm2mins, mins2hhmm, add2hhmm

#import pprint

dp = {}
desc ={}
ats_dir = '/Users/szagar/ZTS/Dropbox/Business/ats'


In [37]:
def connectDB():
    db = Session()
    db_url = os.environ["DB_HEROKU"]

    engine = create_engine(db_url)
    Base.metadata.bind = engine

    DBSession = sessionmaker(bind=engine)
    session = DBSession()
    return session

In [38]:
def nextPrototypeId(dbh):
    #return dbh.query(Prototype).filter(Prototype.id > 0).order_by(Prototype.id).all()
    return dbh.query(ProtoConfig).filter(ProtoConfig.id > 0).order_by(ProtoConfig.id).all()
dbh = connectDB()
print(nextPrototypeId(dbh)[0].id)

1


In [39]:
def queryPrototype(dbh,id):
    return dbh.query(ProtoConfig).filter(ProtoConfig.id == id).all()
    #return dbh.query(Prototype).filter(Prototype.id == id).all()
#dbh = connectDB()
#print(queryPrototype(dbh,1)[0].symbol)

In [40]:
def queryPrototypeParams(dbh,id):
    return dbh.query(PrototypeParameter).filter(PrototypeParameter.proto_id == id).all()
#dbh = connectDB()
#print(queryPrototypeParams(dbh,1)[0].name)


In [6]:
def queryEntryFilter(dbh,id):
    return dbh.query(EntryFilter).filter(EntryFilter.id == id).one_or_none()
#dbh = connectDB()
#print(queryEntryFilter(dbh,1).long_logic)

In [7]:
def queryPoi(dbh,id):
    return dbh.query(Poi).filter(Poi.id == id).one_or_none()
#dbh = connectDB()
#print(queryPoi(dbh,1).long_logic)

In [8]:
def querySession(dbh,id):
    return dbh.query(TradeSession).filter(TradeSession.id==id).one_or_none()
#dbh = connectDB()
#print(f'{querySession(dbh,8).session_start} :: {querySession(dbh,8).session_end}')

In [9]:
'''def defineInputsVars():
    #pattern = re.compile(r'^\w+,\w+,\w+$')
    inputs = {}
    varz = {}
    pattern = re.compile(r'^.+,.+,.+$')
    for i in InputsVars:
        if i in dp:
            match = re.search(pattern, str(dp[i]))
            if match:
                if i not in inputs:
                    inputs[i] = {}
                inputs[i]['optStr'] = dp[i]
                inputs[i]['baseVal'] = 'tbd' #candData[i]
                inputs[i]['dtype'] = lookupDtypes(i)   #varRef[i]
            else:
                if i not in varz:
                    varz[i] = {}
                varz[i]['baseVal'] = dp[i] #candData[i]
                varz[i]['dtype'] = lookupDtypes(i)   #varRef[i]
                #print(f'{i}({dp[i]}) :Not found')
    return inputs, varz
'''

"def defineInputsVars():\n    #pattern = re.compile(r'^\\w+,\\w+,\\w+$')\n    inputs = {}\n    varz = {}\n    pattern = re.compile(r'^.+,.+,.+$')\n    for i in InputsVars:\n        if i in dp:\n            match = re.search(pattern, str(dp[i]))\n            if match:\n                if i not in inputs:\n                    inputs[i] = {}\n                inputs[i]['optStr'] = dp[i]\n                inputs[i]['baseVal'] = 'tbd' #candData[i]\n                inputs[i]['dtype'] = lookupDtypes(i)   #varRef[i]\n            else:\n                if i not in varz:\n                    varz[i] = {}\n                varz[i]['baseVal'] = dp[i] #candData[i]\n                varz[i]['dtype'] = lookupDtypes(i)   #varRef[i]\n                #print(f'{i}({dp[i]}) :Not found')\n    return inputs, varz\n"

In [10]:
def render_from_template(directory, template_name, **kwargs):
    loader = FileSystemLoader(directory)
    env = Environment(loader=loader)
    template = env.get_template(template_name)
    return template.render(**kwargs)

In [11]:
def formatCommentStrings(setup):
    #1st timeframe = {setup['timeframe_1']} {setup['timeframe_1_unit']}
    #2nd timeframe = {setup['timeframe_2']} {setup['timeframe_2_unit']}
    #  1st timeframe = {setup['timeframes'][0]['tf']} {setup['timeframes'][0]['unit']}
    #  2nd timeframe = {setup['timeframes'][1]['tf']} {setup['timeframes'][1]['unit']}
    tf_arr = []
    for tf in setup['timeframes']:
        tf_arr.append(f"Timeframe {tf['ds']} = {tf['tf']} {tf['unit']}")
    tf_str = ('\n').join(tf_arr)
    #print(f"formatCommentStrings:tf={tf}")
    desc['chart_setup'] = f"""Chart Setup:
      symbol = {setup['symbol']}
      {tf_str}
      Fitness Function = {setup['fitnessFunc']}
      maxDaysBack = {setup['maxBarsBack']}
      session name = {setup['session_name']}
      start date = {setup['start_dt']}
      end date = {setup['end_dt']}"""
    desc['prototype_info'] = f'''Prototype Info::
      dataSet = {setup['data_set']}
      dataBlock = {setup['data_block']}    
    '''

In [12]:
def addInputRange(p):
    inp_def = {}
    inp_def['type'] = 'range'
    inp_def['dtype'] = p.data_type
    inp_def['params'] = p.value
    if p.data_type == 'double':
        st,end,step = [float(x) for x in p.value.split(',')]
        inp_def['value'] = [format(x,'.2f') for x in np.arange(st,end+step,step)]
    else:
        st,end,step = [int(x) for x in p.value.split(',')]
        inp_def['value'] = [int(x) for x in np.arange(st,end+step,step)]
    return inp_def

In [13]:
def addInputList(p):
    inp_def = {}
    inp_def['type'] = 'list'
    inp_def['dtype'] = p.data_type
    inp_def['value'] = [int(x) for x in p.value.split(',')]
    return inp_def

In [14]:
def addInputVar(p):
    inp_def = {}
    inp_def['type'] = 'var'
    inp_def['dtype'] = p.data_type
    inp_def['value'] = [p.value,]
    return inp_def

In [15]:
def warn(str):
    print(f'WARNING: {str}')

In [16]:
def parseInput(param):
    if param.input_type == 'range':
        return param.name, addInputRange(param)
    elif param.input_type == 'list':
        return param.name, addInputList(param)
    elif param.input_type == 'var':
        return param.name, addInputVar(param)
    else:
        warn(f'input_type({p.input_type}) not coded for, id={p.id}')

In [17]:
def setSessionVars(id):
    sess = querySession(dbh,id)
    return sess.session_start, sess.session_end

In [18]:
def setDataSeries(series):
    # 1:@RTY.D:120:m,2:@RTY.D:1440:m
    ds = []
    for s in series.split(','):
        sn,symbol,tf,tf_unit = s.split(':')
        tf_dict = {'sn': sn, 'symbol': symbol, 'tf': tf, 'tf_unit': tf_unit}
        ds.append(tf_dict)
    return ds

#series = '1:@RTY.D:120:m,2:@RTY.D:1440:m'
#print(setDataSeries(series))

In [19]:
import datetime

def formatDate(from_dt):
    # .SetFirstDate("2015-12-30")             { "12/30/2015" }
    #return datetime.datetime.strftime(datetime.datetime.strptime(from_dt,'%Y-%m-%d'),'%d/%m/%Y')
    return datetime.datetime.strftime(from_dt,'%m/%d/%Y')


        // Security 1 (Data1)
        security = job.Securities.AddSecurity();
        security.Symbol = "AAPL";
        security.Interval.SetMinuteChart(60);
        security.History.SetYearsBack(2);

        // Security 2 (Data2)
        security = job.Securities.AddSecurity();
        security.Symbol = "$VIX.X";
        security.Interval.SetDailyChart();
        security.History.SetYearsBack(2);


In [20]:
def getConfiguration(dbh,id):
    proto = queryPrototype(dbh,id)[0]
    params = queryPrototypeParams(dbh,id)
    setup = {}
    setup['proto_id'] = proto.id
    setup['template_version'] = proto.template_version
    setup['symbol'] = proto.symbol
    setup['data_set'] = proto.data_set
    setup['data_block'] = proto.data_block
    setup['chart_series'] = proto.chart_series
    setup['fitnessFunc'] = proto.fitness_function
    setup['maxBarsBack'] = proto.max_days_back
    setup['oosPercentLast'] = 20
    setup['session_num'] = proto.session_id
    setup['session_name'] = f'session {proto.session_id}'
    setup['start_dt'] = formatDate(proto.start_dt)
    setup['end_dt'] = formatDate(proto.end_dt)
    setup['chart_series'] = proto.chart_series

    setup['sess_start'],setup['sess_end'] = setSessionVars(setup['session_num'])
    setup['data_series'] = setDataSeries(setup['chart_series'])
    
    setup['opt_inputs'] = {}
    setup['param_vars'] = {}
    for p in params:
        param_name,param_setup = parseInput(p)
        setup['opt_inputs'][param_name] = param_setup
    setup['timeframes'] = []
    for ds in setup['chart_series'].split(','):
        i,symbol,tf,unit = ds.split(':')
        tf_d = {'ds': i, 'symbol': symbol, 'tf': tf, 'unit': unit}
        setup['timeframes'].append(tf_d)
    return setup

In [21]:
def setHeader(setup):
    return f'''      prototype: { setup['proto_id'] }
      template: { setup['template_version'] }
      job file: { job_file }
      poi IDs     : { glom(setup,'opt_inputs.poi.value',default=None) }
      filter 1 IDs: { glom(setup,'opt_inputs.filter_1.value',default=None) }
      filter 2 IDs: { glom(setup,'opt_inputs.filter_2.value',default=None) }'''

In [22]:
def filterLogic(setup):
    filters = {}
    ds = 1
    while True:
        ids = glom(setup,f'opt_inputs.filter_{ds}.value',default=None)
        if ids is None:
            break
        if isinstance(ids,str):
            ids = ids.split(',')
        idx = 1
        filters[ds] = {}
        for id in ids:
            q = queryEntryFilter(dbh,id)
            try:
                #print(f"long_code{id}: {q.long_logic}")
                long_code = q.long_logic.format(p1=f'filter_{ds}_n1',p2=f'filter_{ds}_n2',fn=ds)
            except AttributeError:
                long_code = None
            try:
                #print(f"short_code{id}: {q.short_logic}")
                short_code = q.short_logic.format(p1=f'filter_{ds}_n1',p2=f'filter_{ds}_n2',long=f'{long_code}',fn=ds)
            except AttributeError:
                short_code = None

            #d_filter = {'fid': id, 
            #            'long': long_code,
            #            'short': short_code}
            filters[ds][idx] = {'fid': id, 
                        'long': long_code,
                        'short': short_code}
            idx+=1
            
        ds+=1
    return filters

In [23]:
def lsbLogic(setup):
    if glom(setup,'opt_inputs.lsb.value',default=None) != 4: return
    setup['opt_inputs']['lsb']['value'] = 3
    logic = f'''
        switch(dow)
        begin
            case 1:
                lsb = mon_lsb;
            case 2:
                lsb = tue_lsb;
            case 3:
                lsb = wed_lsb;
            case 4:
                lsb = thu_lsb;
            case 5:
                lsb = fri_lsb;
        end;
    '''
    return logic

In [24]:

def poiLogic(setup):
    id_list = glom(setup,'opt_inputs.poi.value',default=None)
    logic = []
    if id_list == '' or id_list == None: return logic
    ids = id_list
    if isinstance(id_list, str):
        ids = id_list.split(',')     
    if len(ids) == 0: return []
    for id in ids:
        q = queryPoi(dbh,id)
        if not q:
            logic = None
            warn(f"poiLogic: No POI logic for id: {id}")
            continue
        #pprint(q)
        logic.append({'poi_id': id, 'common': q.common_logic, 'long': q.long_logic, 'short': q.short_logic})
        if(q.vars):
            for v in q.vars.split(';'):
                name,dtype,value = v.split(':')
                setup['param_vars'][name] = {'d_type': dtype, 'value': value}
                #'opt_inputs': {'filter_1': {'dtype': 'int',
                #             'type': 'distinct',
                #             'value': [2, 5, 6]},
            #pprint(setup)      #'param_vars': 'int:cum_ma_cnt;int:cum_ma',
    return logic

In [25]:
def calcTimeSegmentSize(start,end,n_segs=3):
    return (hhmm2mins(end)-hhmm2mins(start)) / n_segs
#print(calcTimeSegmentSize(930,1600))

In [26]:
def timeSegmentLogic(setup):
    #print(f"timeSegmentLogic(seg): timeSegmentLogic({seg})")
    seg=glom(setup,'opt_inputs.time_segment.value',default=None)
    tseg = calcTimeSegmentSize(setup['sess_start'],setup['sess_end'])
    #timeframe = setup['timeframe_1']
    timeframe = setup['timeframes'][0]['tf']
    sess_start = setup['sess_start']
    sess_end = setup['sess_end']
    if len(seg)>1:
        code = f'''
        Switch (time_segment) begin
          case 0: begin
            time_filter = True;
          end;
          case 1 : begin
            time_filter = time > {sess_start} and time <= {add2hhmm(sess_start, tseg)};
          end;
          case 2 : begin
            time_filter = time > {add2hhmm(sess_start, tseg)} and time <= {add2hhmm(sess_start, (2*tseg) )};
          end;
          case 3 : begin
            time_filter = time > {add2hhmm(sess_start, (2*tseg))} and time < {add2hhmm(sess_end,-timeframe)};
          end;
        end;
        '''
    elif (len(seg) == 1):
        if seg[0] == '0':
            code = f"time_filter = True;"
        elif seg[0] == '1':
            code = f"time_filter = time > {sess_start} and time <= {add2hhmm(sess_start, tseg)};"
        elif seg[0] == '2':
            code = f"time_filter = time > {add2hhmm(sess_start, tseg)} and time <= {add2hhmm(sess_start, (2*tseg) )};"
        elif seg[0] == '3':
            code = f"time_filter = time > {add2hhmm(sess_start, (2*tseg))} and time < {add2hhmm(sess_end,-timeframe)};"
        else:
            warn(f"timeSegmentLogic({seg}) NOT CODED")
    else:
        code = f"time_filter = True;"
        
    return code

#timeframe=15
#setup={}
#setup['sess_start']=930
#setup['sess_end']=1600
#timeSegmentLogic([0])
#timeSegmentLogic([1])
#timeSegmentLogic([2])
#timeSegmentLogic([3])
#timeSegmentLogic([1,2,3])


In [27]:
def stopLossLogic(setup):
    param=glom(setup,'opt_inputs.stop_loss.value',default=None)
    if param == None: return None
    code = "SetStopLoss(stop_loss);"
    return code

In [28]:
def profitTargetLogic(setup):
    param=glom(setup,'opt_inputs.profit_target.value',default=None)
    if param == None: return None
    if setup:
        code = "SetProfitTarget(profit_target);"
    return code

In [29]:
def processTemplate(hdr,setup,logic):
    templateLoader = FileSystemLoader(searchpath=f"{ats_dir}/Code/Templates/Prototype/")
    templateEnv = Environment(loader=templateLoader)
    TEMPLATE_FILE = setup['template_version']
    print(f"processTemplate: template file = {TEMPLATE_FILE}")
    template = templateEnv.get_template(TEMPLATE_FILE)
    outputText = template.render(header=hdr,
                                 chart_setup=desc['chart_setup'],
                                 prototype_info=desc['prototype_info'],
                                 dp_data=dp,
                                 comments=desc,
                                 poi = "more testing for poi",
                                 filters = logic['filters'],
                                 setup=setup,
                                 logic=logic
                                )
    return(outputText)

In [30]:
def processJclTemplate(setup):
    in_sample_file = f"proto_{setup['proto_id']}_is.csv"
    out_of_sample_file = f"proto_{setup['proto_id']}_oos.csv"
    templateLoader = FileSystemLoader(searchpath=f"{ats_dir}/Code/Templates/JCL/")
    templateEnv = Environment(loader=templateLoader)
    TEMPLATE_FILE = f"proto_{setup['proto_id']}.jcl"
    TEMPLATE_FILE = f"jcl_v2"
    print(f"processJclTemplate: template file = {TEMPLATE_FILE}")
    template = templateEnv.get_template(TEMPLATE_FILE)
    outputText = template.render(setup=setup,
                                 in_sample_file=in_sample_file,
                                 out_of_sample_file=out_of_sample_file
                                )
    #outputText = template.render(header="hdr",
    #                             chart_setup=desc['chart_setup'],
    #                             prototype_info=desc['prototype_info'],
    #                             dp_data=dp,
    #                             comments=desc,
    #                             poi = "more testing for poi",
    #                             filters = "logic['filters']",
    #                             setup=setup,
    #                             logic="logic"
    #                            )
    return(outputText)

In [31]:
def generateEL(dbh,setup):
    logic = {}
    hdr = setHeader(setup)
    logic['lsb'] = lsbLogic(setup)
    logic['poi'] = poiLogic(setup)
    logic['filters'] = filterLogic(setup)
    #pprint(logic['filters'])
    logic['tseg'] = timeSegmentLogic(setup)
    logic['stop_loss'] = stopLossLogic(setup)
    logic['profit_target'] = profitTargetLogic(setup)
    formatCommentStrings(setup)
    #print("hdr:")
    #pprint(hdr)
    #print("setuo:")
    #pprint(setup)
    #print("logic:")
    #pprint(logic)
    strat = processTemplate(hdr,setup,logic)
    dir = f'{ats_dir}/Data/PrototypeCode/'
    file_out = f"{dir}{setup['strategy_name']}"
    print(f"write Proto to {file_out}")
    open(file_out, "w").write(strat)
    return strat

In [32]:
def generateJCL(dbh,setup):
    code = processJclTemplate(setup)
    dir = f"{ats_dir}/Data/OptimizationApiCode/"
    file_out = f"{dir}{setup['jcl_file']}"
    print(f"write JCL to {file_out}")
    open(file_out, "w").write(code)
    return code

In [33]:
dbh = connectDB()
proto_id = nextPrototypeId(dbh)[0].id
setup = getConfiguration(dbh,proto_id)
setup['strategy_name'] = f"proto_{setup['proto_id']}_b2"
setup['jcl_file'] = f"jcl_{setup['proto_id']}_b1.jcl"

In [34]:

job_file = f'opt_job_proto_{proto_id}'
out = generateEL(dbh,setup)
#print(out)


processTemplate: template file = v4
write Proto to /Users/szagar/ZTS/Dropbox/Business/ats/Data/PrototypeCode/proto_1_b2


In [35]:
#pprint(setup)
out = generateJCL(dbh,setup)
#print(out)


processJclTemplate: template file = jcl_v2
write JCL to /Users/szagar/ZTS/Dropbox/Business/ats/Data/OptimizationApiCode/jcl_1_b1.jcl
